![arangodb](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1)

<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/AqlPart2Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AQL Tutorial Part 2

In the [first part of this tutorial](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlCrudTutorial.ipynb) we have looked at basic CRUD queries. In this second part, we will look at some more advanced AQL features:

* Filtering 
* Sorting
* Limiting


Recall, that in addition to the python interface, you can interact with ArangoDB using its web interface to manage collections and execute the queries.

# Setup 

Before getting started with ArangoDB we need to prepare our environment and create a temporary database on ArangoDB's managed Service Oasis.

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import json
import requests
import sys
import oasis
import time

from pyArango.connection import *
from arango import ArangoClient

Create the temporary database:

In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AqlPart2Tutorial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login["dbName"]] 

Requesting new temp credentials.
Temp database ready to use.


In [4]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUT08ov1y4zr7lqbzjazxfn54l
Password: TUTs85q9qv6qzlaqxmyg8bvze
Database: TUTmhrp5uc2yude9nm189h7


Feel free to use to above URL to checkout the UI!

##  Import Data

We will work with the same dataset as in the [first part of this tutorial](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlCrudTutorial.ipynb). So let us quickly import that into our temporary database. 

In [5]:
# Create the Characters Collection 
db.createCollection(name="Characters")

ArangoDB collection name: Characters, id: 16021974, type: document, status: loaded

In [6]:
insert_query = """
LET data = [
    { "name": "Ned", "surname": "Stark", "alive": true, "age": 41, "traits": ["A","H","C","N","P"] },
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

As before let us check the `Characters` collection:

In [7]:
all_characters_names = """
FOR c IN Characters
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Ned

Robert

Jaime

Catelyn

Cersei

Daenerys

Jorah

Petyr

Viserys

Jon

Sansa

Arya

Robb

Theon

Bran

Joffrey

Sandor

Tyrion

Khal

Tywin

Davos

Samwell

Stannis

Melisandre

Margaery

Jeor

Bronn

Varys

Shae

Talisa

Gendry

Ygritte

Tormund

Gilly

Brienne

Ramsay

Ellaria

Daario

Missandei

Tommen

Jaqen

Roose

The High Sparrow



# Filter

To find documents that fulfill certain criteria more complex than key equality, there is the FILTER operation in AQL, which enables us to formulate arbitrary conditions for documents to match.
We actually have used a filter condition before to find `Ned`:

In [8]:
find_ned_query = """
FOR c IN Characters
    FILTER c.name == "Ned"
    RETURN c
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'_key': '1026024985', '_id': 'Characters/1026024985', '_rev': '_apllrXK---', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}



The filter condition reads like: “the attribute name of a character document must be equal to the string Ned”. If the condition applies, character document gets returned. This works with any attribute likewise:

In [9]:
find_ned_query = """
FOR c IN Characters
    FILTER c.surname == "Stark"
    RETURN c
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'_key': '1026024985', '_id': 'Characters/1026024985', '_rev': '_apllrXK---', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}

{'_key': '1026024988', '_id': 'Characters/1026024988', '_rev': '_apllrXK--B', 'name': 'Catelyn', 'surname': 'Stark', 'alive': False, 'age': 40, 'traits': ['D', 'H', 'C']}

{'_key': '1026024995', '_id': 'Characters/1026024995', '_rev': '_apllrXK--J', 'name': 'Sansa', 'surname': 'Stark', 'alive': True, 'age': 13, 'traits': ['D', 'I', 'J']}

{'_key': '1026024996', '_id': 'Characters/1026024996', '_rev': '_apllrXK--K', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 11, 'traits': ['C', 'K', 'L']}

{'_key': '1026024997', '_id': 'Characters/1026024997', '_rev': '_apllrXK--L', 'name': 'Robb', 'surname': 'Stark', 'alive': False, 'traits': ['A', 'B', 'C', 'K']}

{'_key': '1026024999', '_id': 'Characters/1026024999', '_rev': '_apllrXK--N', 'name': 'Bran', 'surname': 'Stark', 'alive': True, 'age': 10, 'traits': [

## Range conditions
Strict equality is one possible condition we can state. There are plenty of other conditions we can formulate however. For example, we could ask for all young characters:

In [10]:
find_adults_query = """
FOR c IN Characters
    FILTER c.age >= 13
    RETURN c.name
"""
query_result = db.AQLQuery(find_adults_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Ned

Jaime

Catelyn

Cersei

Daenerys

Jon

Sansa

Theon

Joffrey

Tyrion

Davos

Samwell

Brienne



The operator >= stands for greater-or-equal, so every character of age 13 or older is returned (only their name in the example). We can return names and age of all characters younger than 13 by changing the operator to less-than and using the object syntax to define a subset of attributes to return:

In [11]:
find_young_query = """
FOR c IN Characters
    FILTER c.age < 13
    RETURN { name: c.name, age: c.age }
"""
query_result = db.AQLQuery(find_young_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()   

{'name': 'Robert', 'age': None}

{'name': 'Jorah', 'age': None}

{'name': 'Petyr', 'age': None}

{'name': 'Viserys', 'age': None}

{'name': 'Arya', 'age': 11}

{'name': 'Robb', 'age': None}

{'name': 'Bran', 'age': 10}

{'name': 'Sandor', 'age': None}

{'name': 'Khal', 'age': None}

{'name': 'Tywin', 'age': None}

{'name': 'Stannis', 'age': None}

{'name': 'Melisandre', 'age': None}

{'name': 'Margaery', 'age': None}

{'name': 'Jeor', 'age': None}

{'name': 'Bronn', 'age': None}

{'name': 'Varys', 'age': None}

{'name': 'Shae', 'age': None}

{'name': 'Talisa', 'age': None}

{'name': 'Gendry', 'age': None}

{'name': 'Ygritte', 'age': None}

{'name': 'Tormund', 'age': None}

{'name': 'Gilly', 'age': None}

{'name': 'Ramsay', 'age': None}

{'name': 'Ellaria', 'age': None}

{'name': 'Daario', 'age': None}

{'name': 'Missandei', 'age': None}

{'name': 'Tommen', 'age': None}

{'name': 'Jaqen', 'age': None}

{'name': 'Roose', 'age': None}

{'name': 'The High Sparrow', 'age': None}



You may notice that it returns name and age of 30 characters, most with an age of null. The reason for this is, that null is the fallback value if an attribute is requested by the query, but no such attribute exists in the document, and the null is compares to numbers as lower (see [Type and value order](https://www.arangodb.com/docs/stable/aql/fundamentals-type-value-order.html)). Hence, it accidentally fulfills the age criterion c.age < 13 (null < 13). To not let documents pass the filter without an age attribute, we can add a second criterion:

In [12]:
find_young_query = """
FOR c IN Characters
    FILTER c.age < 13
    FILTER c.age != null
    RETURN { name: c.name, age: c.age }
"""
query_result = db.AQLQuery(find_young_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()   

{'name': 'Arya', 'age': 11}

{'name': 'Bran', 'age': 10}



This could equally be written with a boolean AND operator as:

In [13]:
find_young_query = """
FOR c IN Characters
    FILTER c.age < 13 AND c.age != null
    RETURN { name: c.name, age: c.age }
"""
query_result = db.AQLQuery(find_young_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()  

{'name': 'Arya', 'age': 11}

{'name': 'Bran', 'age': 10}



If you want documents to fulfill one or another condition, possibly for different attributes as well, use OR:

In [14]:
find_joffrey_query = """
FOR c IN Characters
    FILTER c.name == "Jon" OR c.name == "Joffrey"
    RETURN { name: c.name, surname: c.surname }
"""
query_result = db.AQLQuery(find_joffrey_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()  

{'name': 'Jon', 'surname': 'Snow'}

{'name': 'Joffrey', 'surname': 'Baratheon'}



To learn more about Filter Operation check the [documentation](https://www.arangodb.com/docs/stable/aql/operations-filter.html).

# Sorting

In [15]:
all_characters_names = """
FOR c IN Characters
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Ned

Robert

Jaime

Catelyn

Cersei

Daenerys

Jorah

Petyr

Viserys

Jon

Sansa

Arya

Robb

Theon

Bran

Joffrey

Sandor

Tyrion

Khal

Tywin

Davos

Samwell

Stannis

Melisandre

Margaery

Jeor

Bronn

Varys

Shae

Talisa

Gendry

Ygritte

Tormund

Gilly

Brienne

Ramsay

Ellaria

Daario

Missandei

Tommen

Jaqen

Roose

The High Sparrow



If we recall the above query, the order in which records were returned by the queries shown until is basically random. To return them in a defined order, we can add a SORT() operation.

In [16]:
all_characters_names = """
FOR c IN Characters
    SORT c.name
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Arya

Bran

Brienne

Bronn

Catelyn

Cersei

Daario

Daenerys

Davos

Ellaria

Gendry

Gilly

Jaime

Jaqen

Jeor

Joffrey

Jon

Jorah

Khal

Margaery

Melisandre

Missandei

Ned

Petyr

Ramsay

Robb

Robert

Roose

Samwell

Sandor

Sansa

Shae

Stannis

Talisa

The High Sparrow

Theon

Tommen

Tormund

Tyrion

Tywin

Varys

Viserys

Ygritte



See how it sorted by name, then returned the ten alphabetically first coming names. We can reverse the sort order with DESC like descending:

In [17]:
all_characters_names = """
FOR c IN Characters
    SORT c.name DESC
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Ygritte

Viserys

Varys

Tywin

Tyrion

Tormund

Tommen

Theon

The High Sparrow

Talisa

Stannis

Shae

Sansa

Sandor

Samwell

Roose

Robert

Robb

Ramsay

Petyr

Ned

Missandei

Melisandre

Margaery

Khal

Jorah

Jon

Joffrey

Jeor

Jaqen

Jaime

Gilly

Gendry

Ellaria

Davos

Daenerys

Daario

Cersei

Catelyn

Bronn

Brienne

Bran

Arya



The first sort was ascending, which is the default order. Because it is the default, it is not required to explicitly ask for ASC order.

## Sort by multiple attributes
Assume we want to sort by surname. Many of the characters share a surname. The result order among characters with the same surname is undefined. We can first sort by surname, then name to determine the order:



In [18]:
all_characters_names = """
FOR c IN Characters
    FILTER c.surname
    SORT c.surname, c.name
    LIMIT 10
    RETURN {
        surname: c.surname,
        name: c.name
    }
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'surname': 'Baelish', 'name': 'Petyr'}

{'surname': 'Baratheon', 'name': 'Joffrey'}

{'surname': 'Baratheon', 'name': 'Robert'}

{'surname': 'Baratheon', 'name': 'Stannis'}

{'surname': 'Baratheon', 'name': 'Tommen'}

{'surname': 'Bolton', 'name': 'Ramsay'}

{'surname': 'Bolton', 'name': 'Roose'}

{'surname': 'Clegane', 'name': 'Sandor'}

{'surname': 'Drogo', 'name': 'Khal'}

{'surname': 'Giantsbane', 'name': 'Tormund'}



Overall, the documents are sorted by last name. If the surname is the same for two characters, the name values are compared and the result sorted.



# Limiting

It may not always be necessary to return all documents, that a FOR loop would normally return. In those cases, we can limit the amount of documents with a LIMIT() operation:

In [19]:
sample_chars_query = """
FOR c IN Characters
    SORT c.name
    LIMIT 5
    RETURN c.name
"""
query_result = db.AQLQuery(sample_chars_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()  

Arya

Bran

Brienne

Bronn

Catelyn



LIMIT is followed by a number for the maximum document count. There is a second syntax however, which allows you to skip a certain amount of record and return the next n documents:

In [20]:
sample_chars_query = """
FOR c IN Characters
    SORT c.name
    LIMIT 2, 5
    RETURN c.name
"""
query_result = db.AQLQuery(sample_chars_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()  

Brienne

Bronn

Catelyn

Cersei

Daario



# Sort and Limit Combined

The order can also be determined by a numeric value, such as the age:
A filter is applied to avoid documents without age attribute. The remaining documents are sorted by age in ascending order, and the name and age of the ten youngest characters are returned.

In [21]:
sample_chars_query = """
FOR c IN Characters
    FILTER c.age
    SORT c.age
    LIMIT 10
    RETURN {
        name: c.name,
        age: c.age
    }
"""
query_result = db.AQLQuery(sample_chars_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()  

{'name': 'Bran', 'age': 10}

{'name': 'Arya', 'age': 11}

{'name': 'Sansa', 'age': 13}

{'name': 'Daenerys', 'age': 16}

{'name': 'Jon', 'age': 16}

{'name': 'Theon', 'age': 16}

{'name': 'Samwell', 'age': 17}

{'name': 'Joffrey', 'age': 19}

{'name': 'Tyrion', 'age': 32}

{'name': 'Brienne', 'age': 32}



Please, refer to the documentation for [SORT operation](https://www.arangodb.com/docs/stable/aql/operations-sort.html) and [LIMIT operation](https://www.arangodb.com/docs/stable/aql/operations-limit.html)  for more details.


# Next Steps

To continue playing and working with ArangoDB beyond the temporary database, you can:

* [Get a 2 week free Trial with the ArangoDB Cloud](https://cloud.arangodb.com/home?utm_source=AQLPart2&utm_medium=Github&utm_campaign=ArangoDB%20University)
* [Download ArangoDB](https://www.arangodb.com/download-major/)
* Keep Learning at thttps://www.arangodb.com/arangodb-training-center/

# Further Links

* https://www.arangodb.com/docs/stable/aql/tutorial.html